# Home Credit Prediction: Data Cleaning of Previous Application Table


For an up-to-date version / full view of the plotly - plots, please, go to

Data Cleaning - Previous Application: https://drive.google.com/file/d/16Pl8cB-basjgk0aGTTi7NnlYXGJX7yje/view?usp=sharing

List of all notebooks and resources for this project: https://drive.google.com/file/d/1Z8vPNZAcivWOxeh3UKFfeARbQCMkQ_NR/view?usp=sharing

## Import Modules

In [1]:
%%capture
#! pip install -q kaggle
#! pip install -q pingouin
! pip install -q scikit-optimize
#! pip install -q fastparquet

In [2]:
import numpy as np
import pandas as pd

import sys
import os
import warnings
from importlib import reload

from dask import dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#
#from statsmodels.stats.proportion import proportions_ztest
#import pingouin as pg

from google.colab import drive
drive.mount("/content/gdrive")

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Mounted at /content/gdrive


In [3]:
home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Portfolio/ML_HomeCredit_DefaultRiskEvaluation/'

### Functions

The Python-file with the functions is at
https://drive.google.com/file/d/17IchsTGy2QI9sq0LTIvGvxAk2mrWs4Xz/view?usp=sharing

In [6]:
%load_ext autoreload
%autoreload 2

sys.path.append(home_folder)
import driskfunc as dfunc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load and Update Data

data source: https://storage.googleapis.com/341-home-credit-default/home-credit-default-risk.zip

description: https://storage.googleapis.com/341-home-credit-default/Home%20Credit%20Default%20Risk.pdf

In [7]:
HCdescr = pd.read_csv(home_folder+'data/HomeCredit_columns_description.csv', encoding='latin1') #, dtype=dtype)


In [8]:
HCdescr.loc[HCdescr.Table == 'previous_application.csv']

,Unnamed: 0,Table,Row,Description,Special
173,176,previous_application.csv,SK_ID_PREV,"ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loan applications in Home Credit, previous application could, but not necessarily have to lead to credit)",hashed
174,177,previous_application.csv,SK_ID_CURR,ID of loan in our sample,hashed
175,178,previous_application.csv,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loan [POS] ,...) of the previous application",NaN
176,179,previous_application.csv,AMT_ANNUITY,Annuity of previous application,NaN
177,180,previous_application.csv,AMT_APPLICATION,For how much credit did client ask on the previous application,NaN
178,181,previous_application.csv,AMT_CREDIT,"Final credit amount on the previous application. This differs from AMT_APPLICATION in a way that the AMT_APPLICATION is the amount for which the client initially applied for, but during our approval process he could have received different amount - AMT_CREDIT",NaN
179,182,previous_application.csv,AMT_DOWN_PAYMENT,Down payment on the previous application,NaN
180,183,previous_application.csv,AMT_GOODS_PRICE,Goods price of good that client asked for (if applicable) on the previous application,NaN
181,184,previous_application.csv,WEEKDAY_APPR_PROCESS_START,On which day of the week did the client apply for previous application,NaN
182,185,previous_application.csv,HOUR_APPR_PROCESS_START,Approximately at what day hour did the client apply for the previous application,rounded


In [9]:
HCapp_prev = dd.read_csv(home_folder+'data/previous_application.csv')
df = HCapp_prev
df_name = 'HCapp prev'

In [10]:
df.npartitions

6

In [11]:
#df = df.sort_values(by=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], ascending=False)
df = df.sort_values(by=['SK_ID_CURR','SK_ID_PREV'], ascending=True)

In [12]:
df[['DAYS_DECISION', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE']].head(15)

,DAYS_DECISION,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE
201668,-1740,-1709.0,-1499.0,-1619.0
58558,-606,-565.0,125.0,-25.0
20210,-746,-716.0,-386.0,-536.0
111980,-2341,-2310.0,-1980.0,-1980.0
188131,-828,-797.0,-647.0,-647.0
102029,-815,-784.0,-694.0,-724.0
147347,-315,NaN,NaN,NaN
267213,-757,-706.0,-376.0,-466.0
19368,-438,NaN,NaN,NaN
120987,-181,NaN,NaN,NaN


In [13]:
df.head(15)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
201668,1369693,100001,Consumer loans,3951.000,24835.50,23787.0,2520.00,24835.50,FRIDAY,13,Y,1,0.104326,NaN,NaN,XAP,Approved,-1740,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,23,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
58558,1038818,100002,Consumer loans,9251.775,179055.00,179055.0,0.00,179055.00,SATURDAY,9,Y,1,0.000000,NaN,NaN,XAP,Approved,-606,XNA,XAP,<NA>,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
20210,1810518,100003,Cash loans,98356.995,900000.00,1035882.0,NaN,900000.00,FRIDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
111980,2396755,100003,Consumer loans,6737.310,68809.50,68053.5,6885.00,68809.50,SATURDAY,15,Y,1,0.100061,NaN,NaN,XAP,Approved,-2341,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
188131,2636178,100003,Consumer loans,64567.665,337500.00,348637.5,0.00,337500.00,SUNDAY,17,Y,1,0.000000,NaN,NaN,XAP,Approved,-828,Cash through the bank,XAP,Family,Refreshed,Furniture,POS,XNA,Stone,1400,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
102029,1564014,100004,Consumer loans,5357.250,24282.00,20106.0,4860.00,24282.00,FRIDAY,5,Y,1,0.212008,NaN,NaN,XAP,Approved,-815,Cash through the bank,XAP,Unaccompanied,New,Mobile,POS,XNA,Regional / Local,30,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0
147347,1857999,100005,Cash loans,NaN,0.00,0.0,NaN,NaN,FRIDAY,10,Y,1,NaN,NaN,NaN,XNA,Canceled,-315,XNA,XAP,<NA>,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
267213,2495675,100005,Consumer loans,4813.200,44617.50,40153.5,4464.00,44617.50,THURSDAY,11,Y,1,0.108964,NaN,NaN,XAP,Approved,-757,Cash through the bank,XAP,<NA>,New,Mobile,POS,XNA,Country-wide,37,Connectivity,12.0,high,POS mobile with interest,365243.0,-706.0,-376.0,-466.0,-460.0,0.0
19368,1020698,100006,Cash loans,39954.510,454500.00,481495.5,NaN,454500.00,SATURDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-438,Cash through the bank,XAP,<NA>,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,18.0,high,Cash X-Sell: high,NaN,NaN,NaN,NaN,NaN,NaN
120987,1243599,100006,Cash loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-181,XNA,XAP,<NA>,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
size_df = [df.shape[0].compute(),  df.shape[1]]
print('The', df_name, 'dataset has', size_df[0], 'rows and', size_df[1], 'features.')

The HCapp prev dataset has 1670214 rows and 37 features.


# 2. Data Cleaning

* Handling missing values.
* Removing duplicate samples and features.
* Remove unneccessary columns/rows.
* Treating (here rather checking) the outliers.

## Check Missing Values and Duplicates

Overview of amounts of Nan and of data type:

In [15]:
dfunc.count_dtypes(df, name = df_name)


The dataset HCapp prev has:
16 features of type string.
15 features of type float64.
6 features of type int64.


In [16]:
%%time
%reload_ext autoreload

nan_overview_df = dfunc.nan_type_overview_dd(df, size_df[0])
nan_overview_df_round = nan_overview_df.round(1)
nan_overview_df_round.style.background_gradient(cmap="Blues")

CPU times: user 23.1 s, sys: 3.22 s, total: 26.3 s
Wall time: 20.3 s


,type,NaN[abs],NaN[%]
SK_ID_PREV,int64,0,0.000000
SK_ID_CURR,int64,0,0.000000
NAME_CONTRACT_TYPE,string,0,0.000000
AMT_ANNUITY,float64,372235,22.300000
AMT_APPLICATION,float64,0,0.000000
AMT_CREDIT,float64,1,0.000000
AMT_DOWN_PAYMENT,float64,895844,53.600000
AMT_GOODS_PRICE,float64,385515,23.100000
WEEKDAY_APPR_PROCESS_START,string,0,0.000000
HOUR_APPR_PROCESS_START,int64,0,0.000000


In [17]:
term =  df['CNT_PAYMENT']


### Drop Missing Value Columns

Drop all columns with too many missing values:

In [18]:
#cut_off_percentage = 70.
#
#too_many_nan = nan_overview_df['NaN[%]'
#                ].loc[nan_overview_df['NaN[%]']
#                > cut_off_percentage].round(1)
#
#len(too_many_nan), too_many_nan.index.to_list()

In [19]:
#df = df.drop(columns=too_many_nan.index.to_list())


### Duplicates Check

In [20]:
%reload_ext autoreload

df_dup = dfunc.get_dup_dd(df, name= df_name, size=size_df[0])

Total number of duplicates in " HCapp prev " : 0 ( 0.0 %).


## Other Checks and Modifications

#### Overview

In [21]:
df.describe().compute().T.round(3)

,count,mean,std,min,25%,50%,75%,max
SK_ID_PREV,1670214.0,1923089.135,532597.959,1000001.000,1463955.000,1925331.500,2387346.750,2845382.000
SK_ID_CURR,1670214.0,278357.174,102814.824,100001.000,182114.000,268909.000,359507.000,456255.000
AMT_ANNUITY,1297979.0,15955.121,14782.137,0.000,6359.108,11250.000,20720.261,418058.145
AMT_APPLICATION,1670214.0,175233.860,292779.762,0.000,19035.000,71640.000,183145.500,6905160.000
AMT_CREDIT,1670213.0,196114.021,318574.617,0.000,24255.000,80797.500,219465.000,6905160.000
AMT_DOWN_PAYMENT,774370.0,6697.402,20921.495,-0.900,0.000,1755.000,7852.500,3060045.000
AMT_GOODS_PRICE,1284699.0,227847.279,315396.558,0.000,51255.000,112500.000,236857.500,6905160.000
HOUR_APPR_PROCESS_START,1670214.0,12.484,3.334,0.000,10.000,12.000,15.000,23.000
NFLAG_LAST_APPL_IN_DAY,1670214.0,0.996,0.059,0.000,1.000,1.000,1.000,1.000
RATE_DOWN_PAYMENT,774370.0,0.080,0.108,-0.000,0.000,0.060,0.109,1.000


In [22]:
df_obj = df.describe(exclude=np.number).compute().T
df_obj['freq[%]'] = (df_obj['freq'].astype('float')/df_obj['count'].astype('float')*100)
df_obj.round(1)

,unique,count,top,freq,freq[%]
NAME_CONTRACT_TYPE,4,1670214,Cash loans,747553,44.8
WEEKDAY_APPR_PROCESS_START,7,1670214,TUESDAY,255118,15.3
FLAG_LAST_APPL_PER_CONTRACT,2,1670214,Y,1661739,99.5
NAME_CASH_LOAN_PURPOSE,25,1670214,XAP,922661,55.2
NAME_CONTRACT_STATUS,4,1670214,Approved,1036781,62.1
NAME_PAYMENT_TYPE,4,1670214,Cash through the bank,1033552,61.9
CODE_REJECT_REASON,9,1670214,XAP,1353093,81.0
NAME_TYPE_SUITE,7,849809,Unaccompanied,508970,59.9
NAME_CLIENT_TYPE,4,1670214,Repeater,1231261,73.7
NAME_GOODS_CATEGORY,28,1670214,XNA,950809,56.9


#### Inspect Categoricals

In [23]:
df_pd = df.compute()


In [24]:
df_pd['NAME_CONTRACT_TYPE'].value_counts()

,count
NAME_CONTRACT_TYPE,
Cash loans,747553
Consumer loans,729151
Revolving loans,193164
XNA,346


In [25]:
df_pd['FLAG_LAST_APPL_PER_CONTRACT'].value_counts()

,count
FLAG_LAST_APPL_PER_CONTRACT,
Y,1661739
N,8475


In [26]:
YN_map = {'Y': 1, 'N': 0}
df['FLAG_LAST_APPL_PER_CONTRACT'] = df['FLAG_LAST_APPL_PER_CONTRACT'].map(YN_map)

In [27]:
df_pd['NAME_CASH_LOAN_PURPOSE'].value_counts()

,count
NAME_CASH_LOAN_PURPOSE,
XAP,922661
XNA,677918
Repairs,23765
Other,15608
Urgent needs,8412
Buying a used car,2888
Building a house or an annex,2693
Everyday expenses,2416
Medicine,2174


In [28]:
df_pd['NAME_CONTRACT_STATUS'].value_counts()

,count
NAME_CONTRACT_STATUS,
Approved,1036781
Canceled,316319
Refused,290678
Unused offer,26436


In [29]:
df_pd['NAME_PAYMENT_TYPE'].value_counts()

,count
NAME_PAYMENT_TYPE,
Cash through the bank,1033552
XNA,627384
Non-cash from your account,8193
Cashless from the account of the employer,1085


In [30]:
#df['CODE_REJECT_REASON'].value_counts().compute()

In [31]:
df_pd['NAME_TYPE_SUITE'].value_counts()

,count
NAME_TYPE_SUITE,
Unaccompanied,508970
Family,213263
"Spouse, partner",67069
Children,31566
Other_B,17624
Other_A,9077
Group of people,2240


In [32]:
df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].mask((df['NAME_TYPE_SUITE'].isna()), 'NA')

In [33]:
df_pd['NAME_CLIENT_TYPE'].value_counts()

,count
NAME_CLIENT_TYPE,
Repeater,1231261
New,301363
Refreshed,135649
XNA,1941


In [34]:
#df['NAME_GOODS_CATEGORY'].value_counts().compute()

In [35]:
#XNA
#
#
#Mobile
#
#Consumer Electronics
#Computers
#Audio/Video
#Photo / Cinema Equipment
#
#House Construction
#Furniture
#Homewares
#Office Appliances
#Construction Materials
#Gardening
#
#Clothing and Accessories
#Jewelry
#
#
#Auto Accessories
#Vehicles
#
#Medical Supplies
#Medicine
#
#Sport and Leisure
#Fitness
#Tourism
#
#
#Other
#Direct Sales
#Additional Service
#Education
#Weapon
#Insurance
#Animals
#

In [36]:
df_pd['NAME_PORTFOLIO'].value_counts()

,count
NAME_PORTFOLIO,
POS,691011
Cash,461563
XNA,372230
Cards,144985
Cars,425


In [37]:
df_pd['NAME_PRODUCT_TYPE'].value_counts()

,count
NAME_PRODUCT_TYPE,
XNA,1063666
x-sell,456287
walk-in,150261


In [38]:
#df['CHANNEL_TYPE'].value_counts().compute()

In [39]:
df_pd['NAME_SELLER_INDUSTRY'].value_counts()

,count
NAME_SELLER_INDUSTRY,
XNA,855720
Consumer electronics,398265
Connectivity,276029
Furniture,57849
Construction,29781
Clothing,23949
Industry,19194
Auto technology,4990
Jewelry,2709


In [40]:
df_pd['NAME_YIELD_GROUP'].value_counts()

,count
NAME_YIELD_GROUP,
XNA,517215
middle,385532
high,353331
low_normal,322095
low_action,92041


In [41]:
#df['PRODUCT_COMBINATION'].value_counts().compute()

In [42]:
df_pd['WEEKDAY_APPR_PROCESS_START'].value_counts()

,count
WEEKDAY_APPR_PROCESS_START,
TUESDAY,255118
WEDNESDAY,255010
MONDAY,253557
FRIDAY,252048
THURSDAY,249099
SATURDAY,240631
SUNDAY,164751


In [43]:
week_order  ={'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4,
              'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7}

df['WEEKDAY_APPR_PROCESS_START_ordinal'] = df['WEEKDAY_APPR_PROCESS_START'].map(week_order)
df['WEEKDAY_APPR_PROCESS_START_ordinal'].value_counts().compute()

,count
WEEKDAY_APPR_PROCESS_START_ordinal,
2,255118
3,255010
1,253557
5,252048
4,249099
6,240631
7,164751


In [44]:
del df_pd

#### Interest Rate and AMT ratios

In [45]:
df['AMT_ANNUITY_plain'] = df['AMT_CREDIT']/(df['CNT_PAYMENT'])# + 0.0001)
df['AMT_ANNUITY_diff'] = (df['AMT_ANNUITY'] - df['AMT_ANNUITY_plain'])
df['int_rate_plain'] = df['AMT_ANNUITY_diff']/(df['AMT_ANNUITY_plain'])# + 0.0001)
#df['int_rate_plain_diff'] = df['AMT_ANNUITY_diff']/(df['AMT_ANNUITY_plain'])# + 0.0001)

df['int_rate_plain_diff']  = df['int_rate_plain'] - df['RATE_INTEREST_PRIMARY'] #+ df['RATE_DOWN_PAYMENT']*df['int_rate_plain']
df['AMT_ANNUITY_CREDIT_ratio'] = df['AMT_ANNUITY']/(df['AMT_CREDIT'])# + 0.0001)
df['AMT_ANNUITY_plain_ratio'] = df['AMT_ANNUITY_plain']/(df['AMT_ANNUITY'])# + 0.0001)

df['AMT_GOODS_PRICE_CREDIT_ratio'] = df['AMT_GOODS_PRICE']/(df['AMT_CREDIT'])
df['AMT_DOWN_PAYMENT_CREDIT_ratio'] = df['AMT_DOWN_PAYMENT']/(df['AMT_CREDIT'])

df['CNT_PAYMENT_inv'] = 1/ df['CNT_PAYMENT']

df = df.replace([np.inf, -np.inf], np.nan)


Key finding: Annuity/Credit = 10%

In [46]:
df.loc[(df.AMT_ANNUITY>0)&
        (df.AMT_CREDIT>0)&
        (df.AMT_ANNUITY_plain>0)&
        (df.RATE_INTEREST_PRIMARY>0)
        ][['SK_ID_CURR', 'SK_ID_PREV',
           'int_rate_plain', 'int_rate_plain_diff', #'RATE_DOWN_PAYMENT',
           'RATE_INTEREST_PRIMARY',
           'RATE_INTEREST_PRIVILEGED', 'AMT_ANNUITY_plain_ratio', 'AMT_ANNUITY',
           'AMT_ANNUITY_plain', 'AMT_ANNUITY_CREDIT_ratio',
           'AMT_ANNUITY_plain_ratio', 'CNT_PAYMENT_inv',
           'AMT_GOODS_PRICE_CREDIT_ratio',
           'AMT_DOWN_PAYMENT_CREDIT_ratio',

          ]].compute().head(50).round(3)

,SK_ID_CURR,SK_ID_PREV,int_rate_plain,int_rate_plain_diff,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,AMT_ANNUITY_plain_ratio,AMT_ANNUITY,AMT_ANNUITY_plain,AMT_ANNUITY_CREDIT_ratio,AMT_ANNUITY_plain_ratio,CNT_PAYMENT_inv,AMT_GOODS_PRICE_CREDIT_ratio,AMT_DOWN_PAYMENT_CREDIT_ratio
167563,100057,1130787,0.211,0.022,0.189,0.835,0.826,4747.590,3919.875,0.101,0.826,0.083,1.250,0.250
165622,100069,2173424,0.211,0.022,0.189,0.835,0.826,9105.480,7518.000,0.101,0.826,0.083,1.250,0.250
65719,100070,1006389,0.218,0.057,0.161,0.716,0.821,13249.935,10879.500,0.101,0.821,0.083,1.250,0.250
114379,100285,1327833,0.211,0.018,0.193,0.853,0.826,3627.045,2994.675,0.101,0.826,0.083,0.959,0.096
67256,100429,2006549,0.211,0.018,0.193,0.853,0.826,2651.265,2189.040,0.101,0.826,0.083,0.993,0.099
254952,100507,1984509,0.211,0.014,0.197,0.867,0.826,12140.370,10023.750,0.101,0.826,0.083,1.000,0.000
33949,100522,1239912,0.211,0.014,0.197,0.867,0.826,2597.940,2145.000,0.101,0.826,0.083,1.000,0.000
140098,100522,1830475,0.211,0.028,0.183,0.867,0.826,2597.940,2145.000,0.101,0.826,0.083,1.000,0.000
154730,100540,2393240,0.211,0.022,0.189,0.835,0.826,5753.610,4750.500,0.101,0.826,0.083,1.250,0.250
227063,100552,2202926,0.211,0.035,0.176,0.835,0.826,2874.555,2373.375,0.101,0.826,0.083,1.250,0.250


#### dummies

In [47]:
feat_value_gaps = ['NAME_CONTRACT_TYPE', 'NAME_PAYMENT_TYPE', 'NAME_TYPE_SUITE']

for gap_feat in feat_value_gaps:
    renamed = {}
    for key in df[gap_feat].unique():
        renamed[key] = key.replace(' ','_')
    df[gap_feat] = df[gap_feat].replace(renamed)

In [48]:
dummies_to_concat = []
features_to_dummies = ['WEEKDAY_APPR_PROCESS_START', 'NAME_CONTRACT_TYPE',
                       'NAME_CONTRACT_STATUS',
                       'NAME_PAYMENT_TYPE', 'NAME_TYPE_SUITE',
                       'NAME_CLIENT_TYPE', 'NAME_PORTFOLIO',
                       'NAME_PRODUCT_TYPE', 'NAME_YIELD_GROUP',
                       ]

for feature in features_to_dummies:
    df_feat_cat = df.categorize(columns = [feature])[feature]
    df_feat_cat_dummy = dd.get_dummies(df_feat_cat, prefix = feature, dtype=int)
    dummies_to_concat.append(df_feat_cat_dummy)

df_bundle_dummies = dd.concat(dummies_to_concat, axis=1)
#df_bundle_dummies.head()

## Merge POS CASH, Credit Card and  Intallment table into common columns (especially aggregates)

In [49]:
HCapp_installments = dd.read_csv(home_folder+'cleaned/HC_installment_payments_cleaned.csv')
HCapp_pos_cash = dd.read_csv(home_folder+'cleaned/HC_POS_CASH_cleaned.csv')
HCapp_credit_card = dd.read_csv(home_folder+'cleaned/HC_credit_card_cleaned.csv')
#HCapp_bureau = dd.read_csv(home_folder+'cleaned/HC_bureau_cleaned.csv')

HCapp_installments['flag_table_install'] = 1
HCapp_pos_cash['flag_table_pos_cash'] = 1
HCapp_credit_card['flag_table_credit_card'] = 1
#HCapp_bureau['flag_table_bureau'] = 1

In [50]:
HCapp_installments = HCapp_installments.drop(columns = 'SK_ID_CURR')

In [51]:
credit_card_drops = ['SK_ID_CURR', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL',
                     'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT',
                     'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT',
                     'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT',
                     'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL',
                     'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
                     'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
                     'CNT_DRAWINGS_POS_CURRENT'] #, 'CNT_INSTALMENT_MATURE_CUM']

HCapp_credit_card = HCapp_credit_card.drop(columns = credit_card_drops)


In [52]:
pos_cash_drops = ['SK_ID_CURR', #'CNT_INSTALMENT_max',
                  'CNT_INSTALMENT_FUTURE_latest']
# CNT_INSTALMENT_max = CNT_PAYMENT in POS CASH
# CNT_INSTALMENT_FUTURE ---> INSTALMENTS_paid

HCapp_pos_cash = HCapp_pos_cash.drop(columns = pos_cash_drops)

In [53]:
#HCapp_target = dd.read_csv(home_folder+'data/application_train.csv',
#                           names=['SK_ID_CURR', 'TARGET'])

In [54]:
df_hook_PREV = df[['SK_ID_CURR', 'SK_ID_PREV']]

In [55]:
len(HCapp_installments), len(HCapp_pos_cash), len(HCapp_credit_card), #len(HCapp_bureau)

(997752, 936325, 104307)

In [56]:
size_df[0]

1670214

In [57]:
936325 + 104307 # pos + card

1040632

In [58]:
#HCapp_pos_cash = HCapp_pos_cash.drop(columns = 'SK_ID_CURR')
#HCapp_installments = HCapp_installments.drop(columns = 'SK_ID_CURR')


In [59]:

df_merge_pos = df_hook_PREV.merge(HCapp_pos_cash, on = ['SK_ID_PREV'],
                                 how = 'inner')


df_merge_pos_install = df_merge_pos.merge(HCapp_installments,
                                          on = ['SK_ID_PREV'],
                                          how = 'outer')

df_merge_pos_install = df_merge_pos_install.loc[df_merge_pos_install.flag_table_pos_cash==1]

In [60]:
#df_merge_pos_install.columns

In [61]:
len(df_merge_pos), len(df_merge_pos_install)

(898903, 898903)

In [62]:
#HCapp_credit_card = HCapp_credit_card.drop(columns = 'SK_ID_CURR')

In [63]:
df_merge_card = df_hook_PREV.merge(HCapp_credit_card, on = ['SK_ID_PREV'],
                                 how = 'inner')

df_merge_card_install = df_merge_card.merge(HCapp_installments,
                                          on = ['SK_ID_PREV'],
                                          how = 'outer')

df_merge_card_install = df_merge_card_install.loc[df_merge_card_install.flag_table_credit_card==1]

In [64]:
#df_merge_card_install.columns

In [65]:
len(df_merge_card), len(df_merge_card_install)

(92935, 92935)

In [66]:
df_card_pos = dd.concat([df_merge_pos_install, df_merge_card_install], axis=0)
df_card_pos = df_card_pos.sort_values(by=['SK_ID_CURR','SK_ID_PREV'], ascending=True)

In [67]:
#len(df_merge_pos_install), len(df_merge_card_install)

In [68]:
##df_with_dummies = df_with_dummies.drop(columns='CREDIT_ACTIVE_red')


In [69]:
len(df_card_pos)

991838

In [70]:
df_card_pos.head().round(2)

,SK_ID_CURR,SK_ID_PREV,SK_DPD,SK_DPD_DEF,INSTALMENTS_paid,STATUS_Active,STATUS_Completed,STATUS_Demand,STATUS_Initialized,MONTHS_BALANCE_count,CNT_INSTALMENT_max,MONTHS_BALANCE_latest,NAME_CONTRACT_STATUS_latest,NAME_CONTRACT_STATUS_red_latest,flag_table_pos_cash,NUM_INSTALMENT_NUMBER_max,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_VERSION_min,DAYS_INSTALMENT_DIFF_mean,AMT_INSTALMENT_DIFF_mean,DAYS_INSTALMENT_DIFF_pos_sum,AMT_INSTALMENT_DIFF_pos_sum,flag_table_install,INSTALMENTS_paid_latest,flag_table_credit_card
125766,100001.0,1369693,0.0,0.0,2.00,0.80,0.20,0.0,0.0,5.0,4.0,-53.0,Completed,Completed,1.0,4.0,1.0,2.0,1.0,-15.50,0.0,0.0,0.0,1.0,NaN,NaN
126331,100002.0,1038818,0.0,0.0,9.00,1.00,0.00,0.0,0.0,19.0,24.0,-1.0,Active,Active,1.0,19.0,1.0,2.0,1.0,-20.42,0.0,0.0,0.0,1.0,NaN,NaN
125469,100003.0,1810518,0.0,0.0,3.50,0.88,0.12,0.0,0.0,8.0,12.0,-18.0,Completed,Completed,1.0,7.0,1.0,2.0,1.0,-4.43,0.0,0.0,0.0,1.0,NaN,NaN
125470,100003.0,2396755,0.0,0.0,5.50,1.00,0.00,0.0,0.0,12.0,12.0,-66.0,Active,Active,1.0,12.0,1.0,1.0,1.0,-6.75,0.0,0.0,0.0,1.0,NaN,NaN
125863,100003.0,2636178,0.0,0.0,3.38,0.88,0.12,0.0,0.0,8.0,6.0,-21.0,Completed,Completed,1.0,6.0,1.0,1.0,1.0,-11.17,0.0,0.0,0.0,1.0,NaN,NaN


## Aggregate by SK_ID_CURR

In [71]:
size_now=len(df)

In [72]:
size_now

1670214

In [73]:
df_with_dummies = dd.concat([df, df_bundle_dummies], axis=1)

#df_merge_card = df_with_dummies.merge(HCapp_credit_card, on = ['SK_ID_PREV', 'SK_ID_CURR'],
#                                 how = 'inner')
df_card_pos = df_card_pos.drop(columns='SK_ID_CURR')

df_dummies_card_pos = df_with_dummies.merge(df_card_pos,
                                          on = ['SK_ID_PREV'],
                                          how = 'left')

#df_dummies_card_pos = df_dummies_card_pos.loc[~(df_dummies_card_pos.NAME_CONTRACT_TYPE.isna())]

df_dummies_card_pos = df_dummies_card_pos.sort_values(by=['SK_ID_CURR','SK_ID_PREV'], ascending=True)

In [74]:
size_then=len(df_dummies_card_pos)

In [75]:
size_then

1670214

In [76]:
df_dummies_card_pos[['MONTHS_BALANCE_count', 'NUM_INSTALMENT_NUMBER_max', 'INSTALMENTS_paid', 'CNT_PAYMENT']].head(20)

,MONTHS_BALANCE_count,NUM_INSTALMENT_NUMBER_max,INSTALMENTS_paid,CNT_PAYMENT
76679,5.0,4.0,2.000000,8.0
96218,19.0,19.0,9.000000,24.0
76494,8.0,7.0,3.500000,12.0
76495,12.0,12.0,5.500000,12.0
76790,8.0,6.0,3.375000,6.0
96219,4.0,3.0,1.500000,4.0
76411,NaN,NaN,NaN,NaN
76791,11.0,9.0,4.500000,12.0
96220,NaN,NaN,NaN,18.0
76733,NaN,NaN,NaN,NaN


In [77]:
df_dummies_card_pos.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,WEEKDAY_APPR_PROCESS_START_ordinal,AMT_ANNUITY_plain,AMT_ANNUITY_diff,int_rate_plain,int_rate_plain_diff,AMT_ANNUITY_CREDIT_ratio,AMT_ANNUITY_plain_ratio,AMT_GOODS_PRICE_CREDIT_ratio,AMT_DOWN_PAYMENT_CREDIT_ratio,CNT_PAYMENT_inv,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,NAME_CONTRACT_TYPE_Cash_loans,NAME_CONTRACT_TYPE_Consumer_loans,NAME_CONTRACT_TYPE_Revolving_loans,NAME_CONTRACT_TYPE_XNA,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_PAYMENT_TYPE_Cash_through_the_bank,NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,NAME_PAYMENT_TYPE_Non-cash_from_your_account,NAME_PAYMENT_TYPE_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group_of_people,NAME_TYPE_SUITE_NA,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse,_partner",NAME_TYPE_SUITE_Unaccompanied,NAME_CLIENT_TYPE_New,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_PORTFOLIO_Cards,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,SK_DPD,SK_DPD_DEF,INSTALMENTS_paid,STATUS_Active,STATUS_Completed,STATUS_Demand,STATUS_Initialized,MONTHS_BALANCE_count,CNT_INSTALMENT_max,MONTHS_BALANCE_latest,NAME_CONTRACT_STATUS_latest,NAME_CONTRACT_STATUS_red_latest,flag_table_pos_cash,NUM_INSTALMENT_NUMBER_max,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_VERSION_min,DAYS_INSTALMENT_DIFF_mean,AMT_INSTALMENT_DIFF_mean,DAYS_INSTALMENT_DIFF_pos_sum,AMT_INSTALMENT_DIFF_pos_sum,flag_table_install,INSTALMENTS_paid_latest,flag_table_credit_card
76679,1369693,100001,Consumer_loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,1,1,0.104326,NaN,NaN,XAP,Approved,-1740,Cash_through_the_bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,23,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0,5,2973.375,977.625,0.328793,NaN,0.166099,0.752563,1.044079,0.10594,0.125000,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0.0,0.0,2.000,0.800,0.200,0.0,0.0,5.0,4.0,-53.0,Completed,Completed,1.0,4.0,1.0,2.0,1.0,-15.500000,0.0,0.0,0.0,1.0,NaN,NaN
96218,1038818,100002,Consumer_loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,1,1,0.000000,NaN,NaN,XAP,Approved,-606,XNA,XAP,NA,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,6,7460.625,1791.150,0.240080,NaN,0.051670,0.806399,1.000000,0.00000,0.041667,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0.0,0.0,9.000,1.000,0.000,0.0,0.0,19.0,24.0,-1.0,Active,Active,1.0,19.0,1.0,2.0,1.0,-20.421053,0.0,0.0,0.0,1.0,NaN,NaN
76494,1810518,100003,Cash_loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,1,1,NaN,NaN,NaN,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash office

In [78]:
df_mean = df_dummies_card_pos.groupby("SK_ID_CURR").mean(numeric_only=True)
df_mean = df_mean.drop(columns='SK_ID_PREV')

df_mean = df_mean.drop(columns=['flag_table_install',
                                'flag_table_pos_cash',
                                'flag_table_credit_card'
                                ])

df_count = df_dummies_card_pos.groupby("SK_ID_CURR")['SK_ID_PREV'].count().to_frame()
df_count = df_count.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_count'})



df_sums = df_dummies_card_pos.groupby("SK_ID_CURR")['flag_table_install',
                                                    'flag_table_pos_cash',
                                                    'flag_table_credit_card'
                                                   ].sum()

sum_cols = {'flag_table_install' : 'flag_table_install_sum',
            'flag_table_pos_cash' : 'flag_table_pos_cash_sum',
            'flag_table_credit_card' : 'flag_table_credit_card_sum'
            }

df_sums = df_sums.rename(columns=sum_cols)


active = df_dummies_card_pos.loc[df_dummies_card_pos.NAME_CONTRACT_STATUS_red_latest=='Active']
df_sums_amt = active.groupby("SK_ID_CURR")['AMT_ANNUITY',
                                               'AMT_APPLICATION',
                                               'AMT_CREDIT',
                                               'AMT_DOWN_PAYMENT',
                                               'AMT_GOODS_PRICE'
                                               ].sum()

df_sums_amt = df_sums_amt.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_sum_active_prev',
                                          'AMT_APPLICATION': 'AMT_APPLICATION_sum_active',
                                          'AMT_CREDIT': 'AMT_CREDIT_sum_active',
                                          'AMT_DOWN_PAYMENT': 'AMT_DOWN_PAYMENT_sum_active',
                                          'AMT_GOODS_PRICE': 'AMT_GOODS_PRICE_sum_active'
                                          })

df_count_active = active.groupby("SK_ID_CURR")['SK_ID_PREV'].count().to_frame()
df_count_active = df_count_active.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_active_count'})

df_active_final = dd.concat([df_sums_amt, df_count_active], axis=1)

df_all_final = dd.concat([df_mean, df_count, df_sums], axis=1)

df_final = df_all_final.merge(df_active_final, left_index=True,
                                      right_index=True,
                                      how='outer')

df_final = df_final.reset_index()
df_final=df_final.sort_values(by='SK_ID_CURR')

df_final.round(2).head(20)

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,WEEKDAY_APPR_PROCESS_START_ordinal,AMT_ANNUITY_plain,AMT_ANNUITY_diff,int_rate_plain,int_rate_plain_diff,AMT_ANNUITY_CREDIT_ratio,AMT_ANNUITY_plain_ratio,AMT_GOODS_PRICE_CREDIT_ratio,AMT_DOWN_PAYMENT_CREDIT_ratio,CNT_PAYMENT_inv,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,NAME_CONTRACT_TYPE_Cash_loans,NAME_CONTRACT_TYPE_Consumer_loans,NAME_CONTRACT_TYPE_Revolving_loans,NAME_CONTRACT_TYPE_XNA,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_PAYMENT_TYPE_Cash_through_the_bank,NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,NAME_PAYMENT_TYPE_Non-cash_from_your_account,NAME_PAYMENT_TYPE_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group_of_people,NAME_TYPE_SUITE_NA,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse,_partner",NAME_TYPE_SUITE_Unaccompanied,NAME_CLIENT_TYPE_New,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_PORTFOLIO_Cards,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,SK_DPD,SK_DPD_DEF,INSTALMENTS_paid,STATUS_Active,STATUS_Completed,STATUS_Demand,STATUS_Initialized,MONTHS_BALANCE_count,CNT_INSTALMENT_max,MONTHS_BALANCE_latest,NUM_INSTALMENT_NUMBER_max,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_VERSION_min,DAYS_INSTALMENT_DIFF_mean,AMT_INSTALMENT_DIFF_mean,DAYS_INSTALMENT_DIFF_pos_sum,AMT_INSTALMENT_DIFF_pos_sum,INSTALMENTS_paid_latest,SK_ID_PREV_count,flag_table_install_sum,flag_table_pos_cash_sum,flag_table_credit_card_sum,AMT_ANNUITY_sum_active_prev,AMT_APPLICATION_sum_active,AMT_CREDIT_sum_active,AMT_DOWN_PAYMENT_sum_active,AMT_GOODS_PRICE_sum_active,SK_ID_PREV_active_count
0,100001,3951.00,24835.50,23787.00,2520.00,24835.50,13.00,1.0,1.0,0.10,NaN,NaN,-1740.00,23.00,8.00,365243.00,-1709.00,-1499.00,-1619.00,-1612.00,0.00,5.00,2973.38,977.62,0.33,NaN,0.17,0.75,1.04,0.11,0.12,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.0,1.00,0.00,0.00,0.0,1.00,0.0,0.0,0.00,0.00,1.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.0,0.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,2.00,0.80,0.20,0.0,0.00,5.00,4.00,-53.00,4.00,1.0,2.00,1.00,-15.50,0.00,0.00,0.00,NaN,1,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,9251.78,179055.00,179055.00,0.00,179055.00,9.00,1.0,1.0,0.00,NaN,NaN,-606.00,500.00,24.00,365243.00,-565.00,125.00,-25.00,-17.00,0.00,6.00,7460.62,1791.15,0.24,NaN,0.05,0.81,1.00,0.00,0.04,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.0,1.00,0.00,0.00,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.0,1.00,0.0,0.0,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.0,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,9.00,1.00,0.00,0.0,0.00,19.00,24.00,-1.00,19.00,1.0,2.00,1.00,-20.42,0.00,0.00,0.00,NaN,1,1.0,1.0,0.0,9251.78,179055.00,179055.00,0.0,179055.00,1.0
2,100003,56553.99,435436.50,484191.00,3442.50,435436.50,14.67,1.0,1.0,0.05,NaN,NaN,-1305.00,533.00,10.00,365243.00,-1274.33,-1004.33,-1054.33,-1047.33,0.67,6.00,50033.62,6520.36,0.15,NaN,0.13,0.87,0.95,0.05,0.11,0.33,0.00,0.33,0.33,0.00,0.00,0.00,0.33,0.67,0.00,0.0,1.00,0.00,0.00,0.0,0.67,0.0,0.0,0.33,0.00,0.67,0.0,0.00,0.0,0.0,0.00,0.33,0.00,0.67,0.33,0.00,0.00

In [79]:
size_df_final = [df_final.shape[0].compute(),  df_final.shape[1]]

print('The condensed dataset has', size_df_final[0], 'rows and', size_df_final[1], 'features.')
print('Initial size was', size_df[0], 'rows and', size_df[1], 'features.')

The condensed dataset has 338857 rows and 104 features.
Initial size was 1670214 rows and 37 features.


In [80]:
nan_overview_df = dfunc.nan_type_overview_dd(df_final, size_df_final[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

,type,NaN[abs],NaN[%]
SK_ID_CURR,int64,0,0.000000
AMT_ANNUITY,float64,480,0.100000
AMT_APPLICATION,float64,0,0.000000
AMT_CREDIT,float64,0,0.000000
AMT_DOWN_PAYMENT,float64,20104,5.900000
AMT_GOODS_PRICE,float64,1064,0.300000
HOUR_APPR_PROCESS_START,float64,0,0.000000
FLAG_LAST_APPL_PER_CONTRACT,float64,0,0.000000
NFLAG_LAST_APPL_IN_DAY,float64,0,0.000000
RATE_DOWN_PAYMENT,float64,20104,5.900000


# Export

In [81]:
%%capture
! mkdir home_folder+'cleaned/'
df_final.to_csv(home_folder+'cleaned/HC_previous_app_cleaned.csv',
                 index=False, single_file = True)